<a href="https://colab.research.google.com/github/gonzoreyes07/Simulaci-n-1/blob/main/Inventarios.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ejemplo 5.5. Sistema de inventarios

La demanda mensual de cierto producto sigue la siguiente distribución de probabilidad empírica:

$$
\begin{array}{c c | c c | c c}
\hline
\text{Cantidad} & \text{Probabilidad} & \text{Cantidad} & \text{Probabilidad} & \text{Cantidad} & \text{Probabilidad} \\
\hline
35 & 0.010 & 44 & 0.029 & 53 & 0.065 \\
36 & 0.015 & 45 & 0.035 & 54 & 0.060 \\
37 & 0.020 & 46 & 0.045 & 55 & 0.050 \\
38 & 0.020 & 47 & 0.060 & 56 & 0.040 \\
39 & 0.022 & 48 & 0.065 & 57 & 0.030 \\
40 & 0.023 & 49 & 0.070 & 58 & 0.016 \\
41 & 0.025 & 50 & 0.080 & 59 & 0.015 \\
42 & 0.027 & 51 & 0.075 & 60 & 0.005 \\
43 & 0.028 & 52 & 0.070 &  &  \\
\hline
\end{array}
$$


El tiempo de entrega está distribuido de acuerdo a la siguiente función de probabilidad:

$$
\begin{array}{|c|c|c|c|}
\hline
\text{Meses} & 1 & 2 & 3 \\
\hline
\text{Probabilidad} & 0.30 & 0.40 & 0.30 \\
\hline
\end{array}
$$

Los factores estacionales para cada uno de los meses del año son como se muestra a continuación:

$$
\begin{array}{|c|c|c|c|}
\hline
\text{Mes} & \text{Factores estacionales} & \text{Mes} & \text{Factores estacionales} \\
\hline
1 & 1.20 & 7 & 0.80 \\
2 & 1.00 & 8 & 0.90 \\
3 & 0.90 & 9 & 1.00 \\
4 & 0.80 & 10 & 1.20 \\
5 & 0.80 & 11 & 1.30 \\
6 & 0.70 & 12 & 1.40 \\
\hline
\end{array}
$$

La información con respecto a los costos relevantes es la siguiente:

Costo de ordenar = $120/orden

Costo de inventario = $20/unidad/año

Costo de faltante = $50/unidad

Si el inventario inicial se asume en 150 unidades, ¿determine la cantidad óptima a ordenar (q) y el nivel óptimo de reorden (R)?


In [1]:
import numpy as np
import random

In [15]:
# Configuración de la simulación
np.random.seed(42)  # Para reproducibilidad
simulaciones = 100  # Número de simulaciones
n = 12  # Duración de la simulación en meses

In [16]:
# Distribución de la demanda
demandas = [35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60]
prob = [0.010, 0.015, 0.020, 0.020, 0.022, 0.023, 0.025, 0.027, 0.028, 0.029, 0.035, 0.045, 0.060, 0.065, 0.070, 0.080, 0.075, 0.070, 0.065, 0.060, 0.050, 0.040, 0.030, 0.016, 0.015, 0.005]

# Distribución del tiempo de entrega
tiempos = [1, 2, 3]
prob_tiempos = [0.30, 0.40, 0.30]

# Factores estacionales
fac = [1.20, 1.00, 0.90, 0.80, 0.80, 0.70, 0.80, 0.90, 1.00, 1.20, 1.30, 1.40]

# Costos
costo_orden = 120
costo_inventario = 20 / 12  # $1.6667 por unidad por mes
costo_faltante = 50

In [17]:
# Definimos las funciones que utilizaremos:

# Función para generar demanda
def demanda():
    return np.random.choice(demandas, p=prob)

# Función para generar tiempo de entrega
def entrega():
    return np.random.choice(tiempos, p=prob_tiempos)

# Función para simular un año
def inventario_anual(R, q, inventario_inicial=150):
    total_costos = 0
    for _ in range(simulaciones):
        inventario = inventario_inicial
        costo_acum = 0
        pendientes = []  # Creamos una lista del mes de llegada y cantidad

        for mes in range(n):
            # Ajustamos la demanda por los factores estacionales
            demanda_inicial = demanda()
            demanda_ajustada = round(demanda_inicial * fac[mes])

            # Llegadas de órdenes
            inventario_nuevo = inventario
            pendientes = [(t, q) for t, q in pendientes if t > mes]
            for entrega_mes, cantidad in [(t, q) for t, q in pendientes if t == mes]:
                inventario_nuevo += cantidad

            if inventario_nuevo <= R:
                cantidad_orden = q
                tiempo_entrega = entrega()
                pendientes.append((mes + tiempo_entrega, cantidad_orden))
                costo_acum += costo_orden

            # Actualizamos el inventario con la demanda ajustada
            inventario = inventario_nuevo - demanda_ajustada

            # Calculo de costos
            if inventario > 0:
                costo_acum += inventario * costo_inventario
            else:
                costo_acum += abs(inventario) * costo_faltante
                inventario = 0

        total_costos += costo_acum

    return total_costos / simulaciones

In [18]:
# Busqueda de los valores óptimos para R y q
mejor_costo = float('inf')
R_optimo = None
q_optimo = None

In [19]:
# Rangos para R y q
R_valores = range(30, 100, 10)  # Niveles de reorden entre 30 y 90
q_valores = range(50, 150, 10)  # Cantidades a ordenar entre 50 y 140

In [21]:
for R in R_valores:
    for q in q_valores:
        costo_promedio = inventario_anual(R, q)
        if costo_promedio < mejor_costo:
            mejor_costo = costo_promedio
            R_optimo = R
            q_optimo = q

# Imprimimos los resultados obtenidos
print(f"Nivel óptimo de reorden (R): {R_optimo}")
print(f"Cantidad óptima a ordenar (q): {q_optimo}")
print(f"Costo promedio anual: ${mejor_costo:.2f}")

Nivel óptimo de reorden (R): 70
Cantidad óptima a ordenar (q): 80
Costo promedio anual: $22871.87
